### Imports

In [20]:
import sys
import io
import os
import datetime
import numpy as np
import pandas as pd
from googleads import adwords
from google.cloud import bigquery

import warnings

### API Call

In [21]:
output = io.StringIO()

In [22]:
adwords_client = adwords.AdWordsClient.LoadFromStorage(r'''C:\Users\cdageforde\Desktop\ads\pro_BRSOads\Credentials\googleads.yaml''')

Your default encoding, cp65001, is not UTF-8. Please run this script with UTF-8 encoding to avoid errors.


In [23]:
report_downloader = adwords_client.GetReportDownloader(version='v201809')

In [24]:
report_query = (adwords.ReportQueryBuilder()
                  .Select('CampaignName', 'CampaignStatus', 'Date', 'CityCriteriaId', 'CountryCriteriaId',
                          'Device', 'AccountCurrencyCode', 'Clicks', 'Impressions', 
                          'Cost', 'Conversions', 'ConversionValue')
                  .From('GEO_PERFORMANCE_REPORT')
                  .Where('CampaignStatus').In('ENABLED', 'PAUSED')
                  .During('LAST_MONTH')
                  .Build())

In [25]:
report_query.__dict__

{'_awql': 'SELECT CampaignName, CampaignStatus, Date, CityCriteriaId, CountryCriteriaId, Device, AccountCurrencyCode, Clicks, Impressions, Cost, Conversions, ConversionValue FROM GEO_PERFORMANCE_REPORT WHERE CampaignStatus IN ["ENABLED", "PAUSED"] DURING LAST_MONTH'}

In [26]:
report_downloader.DownloadReportWithAwql(
                    report_query, 'CSV', output, skip_report_header=True,
                    skip_column_header=False, skip_report_summary=True)

In [27]:
output.seek(0)

0

### Dataframe

In [28]:
df = pd.read_csv(output)
df.head()

,Campaign,Campaign state,Day,City,Country/Territory,Device,Currency,Clicks,Impressions,Cost,Conversions,Total conv. value
0,HTP Q3 2019 OLV BOSS Count Services: Marchex L...,enabled,2019-07-09,1024279,2840,Tablets with full browsers,USD,0,1,127988,0.0,0.0
1,HTP Q3 2019 OLV BOSS Count Services: Marchex L...,enabled,2019-07-09,1024279,2840,Devices streaming video content to TV screens,USD,0,10,213567,0.0,0.0
2,HTP Q3 2019 OLV BOSS Count Services: Good Bett...,enabled,2019-07-10,1024279,2840,Mobile devices with full browsers,USD,0,26,614376,0.0,0.0
3,HTP Q3 2019 OLV BOSS Count Services: Good Bett...,enabled,2019-07-10,1024279,2840,Devices streaming video content to TV screens,USD,0,19,536812,0.0,0.0
4,HTP Q3 2019 OLV BOSS Count Services: Marchex L...,enabled,2019-07-10,1024157,2840,Computers,USD,0,12,260716,0.0,0.0


In [29]:
df.to_csv('..\Data\criteria_performance.csv', index=False)

### Access BigQuery

In [30]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r'''C:\Users\cdageforde\Desktop\ads\pro_BRSOads\Credentials\my_cred.json'''

In [31]:
bigquery_client = bigquery.Client()
dataset_ref = bigquery_client.dataset('pandas_to_bq') #dataset id of new table

### Delete Table

In [32]:
table_id = 'icrossingdatastorage.pandas_to_bq.p2bq_table'
bigquery_client.delete_table(table_id)
print("Deleted table '{}'.".format(table_id))

Deleted table 'icrossingdatastorage.pandas_to_bq.p2bq_table'.


### Create Table

In [33]:
# Prepares a reference to the table
table_ref = dataset_ref.table('p2bq_table') #enter new table name
schema = []
table = bigquery.Table(table_ref, schema=schema)
table = bigquery_client.create_table(table)
print('new table {} created.'.format(table.table_id))

new table p2bq_table created.


### Upload to BigQuery

In [34]:
filename = r'''C:\Users\cdageforde\Desktop\ads\pro_BRSOads\Data\geo_performance.csv''' #csv file path
dataset_id = 'pandas_to_bq' #same dateset name as above
table_id = 'p2bq_table' #same table name as above

In [35]:
dataset_ref = bigquery_client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

In [36]:
with open(filename, 'rb') as source_file:
 job = bigquery_client.load_table_from_file(
       source_file,
       table_ref,
       location='US',  # Must match the destination dataset location.
       job_config=job_config)  # API request

In [37]:
job.result()  # Waits for table load to complete.

In [38]:
print('Loaded {} rows into {}:{}.'.format(
   job.output_rows, dataset_id, table_id))

Loaded 1600 rows into pandas_to_bq:p2bq_table.
